In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [56]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [57]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [58]:
df = df.groupby("user_id").head(3)

In [59]:
df["content_id_shift1"] = df.groupby("user_id")["content_id"].shift(1)
df["content_id_shift2"] = df.groupby("user_id")["content_id"].shift(2)

df["user_answer_shift1"] = df.groupby("user_id")["user_answer"].shift(1)
df["user_answer_shift2"] = df.groupby("user_id")["user_answer"].shift(2)

df["prior_question_elapsed_time_shift1"] = df.groupby("user_id")["prior_question_elapsed_time"].shift(1)
df["prior_question_elapsed_time_shift2"] = df.groupby("user_id")["prior_question_elapsed_time"].shift(2)

In [60]:
df = df[df["content_id_shift2"].notnull()]

In [61]:
df[df["content_id_shift1"] == 7876].groupby(["content_id_shift2", "user_answer_shift2", "content_id_shift1", "user_answer_shift1"]).size()

content_id_shift2  user_answer_shift2  content_id_shift1  user_answer_shift1
4221.0             1.0                 7876.0             2.0                      1
7900.0             0.0                 7876.0             0.0                   1474
                                                          1.0                    922
                                                          2.0                   4903
                                                          3.0                   5034
                   1.0                 7876.0             0.0                    171
                                                          1.0                    117
                                                          2.0                    278
                                                          3.0                    167
                   2.0                 7876.0             0.0                    191
                                                          1.0            

In [62]:
df[(df["content_id_shift2"] == 7900) & (df["content_id_shift1"] == 7876)].groupby(["content_id_shift2", "user_answer_shift2", "content_id_shift1", "user_answer_shift1"]).size()

content_id_shift2  user_answer_shift2  content_id_shift1  user_answer_shift1
7900.0             0.0                 7876.0             0.0                   1474
                                                          1.0                    922
                                                          2.0                   4903
                                                          3.0                   5034
                   1.0                 7876.0             0.0                    171
                                                          1.0                    117
                                                          2.0                    278
                                                          3.0                    167
                   2.0                 7876.0             0.0                    191
                                                          1.0                    125
                                                          2.0            

In [73]:
df[(df["content_id_shift2"] == 7900) & (df["content_id_shift1"] == 7876)].groupby(["content_id_shift2", "user_answer_shift2", "content_id_shift1", "user_answer_shift1", "content_id", "answered_correctly"]).size()

14876

In [76]:
cond = (df["content_id_shift2"] == 7900) & (df["content_id_shift1"] == 7876) & (df["prior_question_elapsed_time"] < 20000)
df[cond].groupby(["content_id_shift2", "user_answer_shift2", "content_id_shift1", "user_answer_shift1", "answered_correctly"]).size()

content_id_shift2  user_answer_shift2  content_id_shift1  user_answer_shift1  answered_correctly
7900.0             0.0                 7876.0             0.0                 0.0                   172
                                                                              1.0                    62
                                                          1.0                 0.0                    67
                                                                              1.0                    31
                                                          2.0                 0.0                   436
                                                                              1.0                   165
                                                          3.0                 0.0                   594
                                                                              1.0                   449
                   1.0                 7876.0             0.0          

In [88]:
cond = (df["content_id_shift2"] == 7900) & (df["content_id_shift1"] == 7876) & (df["prior_question_elapsed_time"] < 30000)
df[cond].groupby(["content_id_shift2", "user_answer_shift2", "content_id_shift1", "user_answer_shift1", "answered_correctly"]).size()

content_id_shift2  user_answer_shift2  content_id_shift1  user_answer_shift1  answered_correctly
7900.0             0.0                 7876.0             0.0                 0.0                    883
                                                                              1.0                    441
                                                          1.0                 0.0                    569
                                                                              1.0                    237
                                                          2.0                 0.0                   3204
                                                                              1.0                   1369
                                                          3.0                 0.0                   2797
                                                                              1.0                   1880
                   1.0                 7876.0             0.0  

In [80]:
cond = (df["content_id_shift2"] == 7900) & (df["content_id_shift1"] == 7876) & (df["prior_question_elapsed_time_shift1"] > 15000) & (df["prior_question_elapsed_time_shift1"] < 000)
df[cond].groupby(["content_id_shift2", "user_answer_shift2", "content_id_shift1", "user_answer_shift1", "answered_correctly"]).size()

content_id_shift2  user_answer_shift2  content_id_shift1  user_answer_shift1  answered_correctly
7900.0             0.0                 7876.0             0.0                 0.0                    808
                                                                              1.0                    401
                                                          1.0                 0.0                    536
                                                                              1.0                    218
                                                          2.0                 0.0                   2873
                                                                              1.0                   1207
                                                          3.0                 0.0                   2431
                                                                              1.0                   1669
                   1.0                 7876.0             0.0  

In [91]:
df.groupby(["content_id", "content_id_shift1", "content_id_shift2"]).size().sort_values(ascending=False)

content_id  content_id_shift1  content_id_shift2
175         7876.0             7900.0               14815
7922        7860.0             128.0                 1362
5344        5716.0             5692.0                 299
1278        7876.0             7900.0                  23
            175.0              7900.0                  18
                                                    ...  
5880        625.0              5102.0                   1
                               6295.0                   1
            638.0              6317.0                   1
            648.0              6448.0                   1
0           42.0               4919.0                   1
Length: 22666, dtype: int64